In [15]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model
m = gp.Model("qp")

# Create variables
x = m.addVar(ub=10.0, vtype=GRB.INTEGER, name="x")
y = m.addVar(ub=10.0, vtype=GRB.INTEGER, name="y")
z = m.addVar(ub=10.0, vtype=GRB.INTEGER, name="z")

# Set objective: x^2 + x*y + y^2 + y*z + z^2 + 2 x
# obj = x*x + x*y + y*y + y*z + z*z + 2*x
# m.setObjective(obj)

# Add constraint: x + 2 y + 3 z <= 4
m.addConstr(x + 2 * y + 3 * z >= 40, "c0")

# Add constraint: x + y >= 1
m.addConstr(x + y >= 1, "c1")


obj = x*x + x*y + y*y + y*z + z*z + 2*x
m.setObjective(obj)
m.optimize()

for v in m.getVars():
    print('%s %g' % (v.varName, v.x))

print('Obj: %g' % obj.getValue())

# x.vType = GRB.INTEGER
# y.vType = GRB.INTEGER
# z.vType = GRB.INTEGER

m.optimize()

for v in m.getVars():
    print('%s %g' % (v.varName, v.x))

print('Obj: %g' % obj.getValue())

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x6defbed3
Model has 5 quadratic objective terms
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 2e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 520.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Presolved model has 5 quadratic objective terms
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: objective 1.720000e+02, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     172.0000000  172.00000  0.00%     -    0s

Explored 0 nodes (5 simplex ite

In [25]:
# This example considers the following separable, convex problem:
#
#   minimize    f(x) - y + g(z)
#   subject to  x + 2 y + 3 z <= 4
#               x +   y       >= 1
#               x,    y,    z <= 1
#
# where f(u) = exp(-u) and g(u) = 2 u^2 - 4 u, for all real u. It
# formulates and solves a simpler LP model by approximating f and
# g with piecewise-linear functions. Then it transforms the model
# into a MIP by negating the approximation for f, which corresponds
# to a non-convex piecewise-linear function, and solves it again.

import gurobipy as gp
from math import exp


def f(u):
    return exp(-u)

def g(u):
    return 2 * u * u - 4 * u

try:
    # Create a new model
    m = gp.Model()

    # Create variables
    lb = 0.0
    ub = 1.0

    x = m.addVar(lb, ub, name='x')
    y = m.addVar(lb, ub, name='y')
    z = m.addVar(lb, ub, name='z')

    # Set objective for y
    m.setObjective(-y)

    # Add piecewise-linear objective functions for x and z
    npts = 101
    ptu = []
    ptf = []
    ptg = []

    for i in range(npts):
        ptu.append(lb + (ub - lb) * i / (npts - 1))
        ptf.append(f(ptu[i]))
        ptg.append(g(ptu[i]))

    ptu = np.arange(0, 1.01, 0.01).tolist()
    m.setPWLObj(x, ptu, ptf)
    m.setPWLObj(z, ptu, ptg)

    # Add constraint: x + 2 y + 3 z <= 4

    m.addConstr(x + 2 * y + 3 * z <= 4, 'c0')

    # Add constraint: x + y >= 1

    m.addConstr(x + y >= 1, 'c1')

    # Optimize model as an LP

    m.optimize()

    print('IsMIP: %d' % m.IsMIP)
    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))
    print('Obj: %g' % m.ObjVal)
    print('')

    # Negate piecewise-linear objective function for x

    for i in range(npts):
        ptf[i] = -ptf[i]

    m.setPWLObj(x, ptu, ptf)

    # Optimize model as a MIP

    m.optimize()

    print('IsMIP: %d' % m.IsMIP)
    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))
    print('Obj: %g' % m.ObjVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xa44777f5
Model has 2 piecewise-linear objective terms
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.6321206e+00   5.000000e-01   0.000000e+00      0s
       2   -1.9346239e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective -1.934623931e+00
IsMIP: 0
x 0.69
y 0.725
z 0.62
Obj: -1.93462

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model has 2 piecewise-linear objective terms
Variable types: 3 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objectiv

In [28]:
# x 0.69
# y 0.725
# z 0.62
# Obj: -1.93462

# x 0
# y 1
# z 0.666667
# Obj: -3.77773
len(ptf)
ptf

[-1.0,
 -0.9900498337491681,
 -0.9801986733067553,
 -0.9704455335485082,
 -0.9607894391523232,
 -0.951229424500714,
 -0.9417645335842487,
 -0.9323938199059483,
 -0.9231163463866358,
 -0.9139311852712282,
 -0.9048374180359595,
 -0.8958341352965282,
 -0.8869204367171575,
 -0.8780954309205613,
 -0.8693582353988059,
 -0.8607079764250578,
 -0.8521437889662113,
 -0.8436648165963837,
 -0.835270211411272,
 -0.8269591339433623,
 -0.8187307530779818,
 -0.8105842459701871,
 -0.8025187979624785,
 -0.794533602503334,
 -0.7866278610665535,
 -0.7788007830714049,
 -0.7710515858035663,
 -0.7633794943368531,
 -0.7557837414557255,
 -0.7482635675785653,
 -0.7408182206817179,
 -0.7334469562242892,
 -0.7261490370736909,
 -0.7189237334319262,
 -0.7117703227626097,
 -0.7046880897187134,
 -0.697676326071031,
 -0.6907343306373547,
 -0.6838614092123558,
 -0.6770568744981647,
 -0.6703200460356393,
 -0.6636502501363194,
 -0.6570468198150567,
 -0.6505090947233165,
 -0.6440364210831414,
 -0.6376281516217733,
 -0.631

In [21]:
import numpy as np
np.arange(0,1.01,0.01).tolist()

[0.0,
 0.01,
 0.02,
 0.03,
 0.04,
 0.05,
 0.06,
 0.07,
 0.08,
 0.09,
 0.1,
 0.11,
 0.12,
 0.13,
 0.14,
 0.15,
 0.16,
 0.17,
 0.18,
 0.19,
 0.2,
 0.21,
 0.22,
 0.23,
 0.24,
 0.25,
 0.26,
 0.27,
 0.28,
 0.29,
 0.3,
 0.31,
 0.32,
 0.33,
 0.34,
 0.35000000000000003,
 0.36,
 0.37,
 0.38,
 0.39,
 0.4,
 0.41000000000000003,
 0.42,
 0.43,
 0.44,
 0.45,
 0.46,
 0.47000000000000003,
 0.48,
 0.49,
 0.5,
 0.51,
 0.52,
 0.53,
 0.54,
 0.55,
 0.56,
 0.5700000000000001,
 0.58,
 0.59,
 0.6,
 0.61,
 0.62,
 0.63,
 0.64,
 0.65,
 0.66,
 0.67,
 0.68,
 0.6900000000000001,
 0.7000000000000001,
 0.71,
 0.72,
 0.73,
 0.74,
 0.75,
 0.76,
 0.77,
 0.78,
 0.79,
 0.8,
 0.81,
 0.8200000000000001,
 0.8300000000000001,
 0.84,
 0.85,
 0.86,
 0.87,
 0.88,
 0.89,
 0.9,
 0.91,
 0.92,
 0.93,
 0.9400000000000001,
 0.9500000000000001,
 0.96,
 0.97,
 0.98,
 0.99,
 1.0]

In [29]:
import gurobipy as gp
import math

m = gp.Model()
x = m.addVar(name="x", ub=1)
y = m.addVar(name="y", lb=-gp.GRB.INFINITY)

# Create x-points and y-points for approximation of y = x*log(x)
xs = [0.01*i for i in range(101)]
ys = [p*math.log(p) if p != 0 else 0 for p in xs]

# Add piecewise-linear constraint
m.addGenConstrPWL(x, y, xs, ys, "pwl")

# Minimize approximation of y = x*log(x)
m.setObjective(y, gp.GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0xad67b7ea
Model has 1 general constraint
Variable types: 2 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000
Presolve added 1 rows and 99 columns
Presolve time: 0.00s
Presolved: 1 rows, 101 columns, 101 nonzeros
Presolved model has 1 SOS constraint(s)
Found heuristic solution: objective -0.3678733
Variable types: 101 continuous, 0 integer (0 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 16 (of 16 available processors)

Solution count 2: -0.367873 0 

Optimal solution found (tolerance 1.00e-04)
Best objective -3.678733411372e-01, best bound -3.678733411372e-01, gap 0.0000%


In [30]:
x

<gurobi.Var x (value 0.37)>

In [31]:
y

<gurobi.Var y (value -0.36787334113723075)>